# Task1

Index_X = FSR_for_force, FSR_for_coord

Index_y = force, x_coord, y_coord

Data = Splited by Time

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-07-04_21-31-27/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-07-04_21-31-27

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.LSTM', 'fsr_model.CNN_LSTM', 'fsr_model.ANN'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': ['FSR_for_force', 'FSR_for_coord'],
        'index_y': ['force', 'x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_time'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=1000,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-07-05 02:02:11,272] A new study created in memory with name: optuna


In [ ]:
results = tuner.fit()

In [13]:
results = tuner.restore(
    path='/home/seokj/ray_results/FSR_Trainable_2023-07-04_21-31-27',
    trainable=FSR_Trainable
).fit()

Failed to read the results for 3 trials:
- /home/seokj/ray_results/FSR_Trainable_2023-07-04_21-31-27/FSR_Trainable_1e8ec326_856_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_Simple_2023-07-05_01-45-27
- /home/seokj/ray_results/FSR_Trainable_2023-07-04_21-31-27/FSR_Trainable_295022da_260_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_Simple_2023-07-04_23-17-48
- /home/seokj/ray_results/FSR_Trainable_2023-07-04_21-31-27/FSR_Trainable_a59376cb_658_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_Simple_2023-07-05_00-56-49
2023-07-05 02:13:18,878	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-05 02:13:20,029	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-07-05 02:13:20,103	INFO experiment_state.py:435 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2023-07-05 02:13:20,139	WARNING trial_runner.py:418 -- Attempting to resume experiment from /home/seokj/ray_results/FSR_Trainable_2023-07-04_21-31-27. This will ignore any new changes to the specification.
2023-07-05 02:13:20,140	INFO trial_runner.py:422 -- Using the newest experiment state file found within the experiment directory: experiment_state-2023-07-04_21-31-32.json
2023-07-05 02:13:22,756	INFO wandb.py:320 -- Already logged into W&B.
(FSR_Trainable pid=202697) 2023-07-05 02:13:31,773	INFO trainable.py:918 -- Restored on 172.26.215.93 from checkpoint: /tmp/checkpoint_tmp_70b95321f16c41668125b2dcc8c645fd
(FSR_Trainable pid=202697) 2023-07-05 02:13:31,773	INFO trainable.py:927 -- Current state after restoring: {'_iteration': 80, '_timesteps_total': None, '_time_total': 276.3226668834686, '_episodes_total': None}


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
FSR_Trainable_0341bc42,2023-07-05_02-25-13,True,DESKTOP-0P789CI,1,86.5541,3.02447e+08,172.26.215.93,218620,255.161,4.93402,4.93402,4.93402,1688491513,1,0341bc42
FSR_Trainable_039599b4,2023-07-05_02-19-49,True,DESKTOP-0P789CI,8,33.009,5.23182e+07,172.26.215.93,210890,107.868,17.8723,2.21719,17.8723,1688491189,8,039599b4
FSR_Trainable_059f96c3,2023-07-05_02-20-33,True,DESKTOP-0P789CI,1,65.2449,1.14558e+08,172.26.215.93,212304,208.429,2.63713,2.63713,2.63713,1688491233,1,059f96c3
FSR_Trainable_07f9f19c,2023-07-05_02-13-57,True,DESKTOP-0P789CI,4,34.9086,6.16229e+07,172.26.215.93,202702,112.123,14.597,2.46022,14.597,1688490837,4,07f9f19c
FSR_Trainable_086b9c96,2023-07-05_02-29-38,True,DESKTOP-0P789CI,1,62.7991,1.46386e+08,172.26.215.93,224987,194.7,1.41878,1.41878,1.41878,1688491778,1,086b9c96
FSR_Trainable_0dea044e,2023-07-05_02-25-39,True,DESKTOP-0P789CI,4,34.1872,5.39482e+07,172.26.215.93,219094,110.8,8.41184,2.39622,8.41184,1688491539,4,0dea044e
FSR_Trainable_0eae9556,2023-07-05_02-27-29,True,DESKTOP-0P789CI,1,85.9466,2.40901e+16,172.26.215.93,221715,271.478,4.75926,4.75926,4.75926,1688491649,1,0eae9556
FSR_Trainable_0f6bcd17,2023-07-05_02-26-56,True,DESKTOP-0P789CI,1,87.5111,2.72192e+08,172.26.215.93,220927,259.712,4.03419,4.03419,4.03419,1688491616,1,0f6bcd17
FSR_Trainable_0ff9f9ef,2023-07-05_02-33-14,True,DESKTOP-0P789CI,8,34.1913,5.50509e+07,172.26.215.93,229459,109.497,18.6416,3.00204,18.6416,1688491994,8,0ff9f9ef
FSR_Trainable_10492b40,2023-07-05_02-15-31,True,DESKTOP-0P789CI,2,43.5909,6.34338e+07,172.26.215.93,205468,140.774,8.48554,3.84757,8.48554,1688490931,2,10492b40


2023-07-05 02:13:42,051	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 3.498 s, which may be a performance bottleneck.
2023-07-05 02:13:42,054	WARNING util.py:315 -- The `process_trial_result` operation took 3.502 s, which may be a performance bottleneck.
2023-07-05 02:13:42,056	WARNING util.py:315 -- Processing trial results took 3.504 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-07-05 02:13:42,058	WARNING util.py:315 -- The `process_trial_result` operation took 3.506 s, which may be a performance bottleneck.
/home/seokj/workspace/.venv/lib/python3.10/site-packages/optuna/trial/_trial.py:494: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
(_WandbLoggingActor pid=202922) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(FSR_Trainable pid=202700) 2023-07-05 02:13:32,581	INFO trainable.py:918 -- Restored on 172